In [1]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm

We will use the story of alice and will generate a sentence similar to Alice

In [2]:
class Dictionary(object):
    def __init__(self):
        # We will have two dictionaries
        self.wordToIndex = {}
        self.indexToWord = {}
        self.index = 0
    
    def add_word(self, word):
        if word not in self.wordToIndex:
            self.wordToIndex[word] = self.index
            self.indexToWord[self.index] = word
            self.index += 1
            
    def __len__(self):
        return len(self.wordToIndex)

In [3]:
class TextProcess(object):
    def __init__(self):
        self.dictionary = Dictionary()
        
    def get_data(self, path, batch_size=20):
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
                    
        rep_tensor = torch.LongTensor(tokens)
        index = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.wordToIndex[word]
                    index += 1
        # Find out how many batches we need
        num_batches = rep_tensor.shape[0] // batch_size
        rep_tensor = rep_tensor[:num_batches * batch_size]
        # return {batch_size, num_batches}
        rep_tensor = rep_tensor.view(batch_size, -1)
        return rep_tensor

In [4]:
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 20
batch_size = 20
timesteps = 30
learning_rate = 0.002

In [5]:
corpus = TextProcess()

In [6]:
rep_tensor = corpus.get_data("./src/alice.txt", batch_size)

In [7]:
rep_tensor.shape

torch.Size([20, 1484])

In [8]:
vocab = corpus.dictionary

In [9]:
len(vocab)

5290

In [10]:
num_batches = rep_tensor.shape[1] // timesteps
print(num_batches)

49


In [15]:
class TextGenerator(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        # Perform Word Embeding
        x = self.embed(x)
        
        # Reshape the input tensor
        #   x = x.view(batch_size, timesteps, embed_size)
        out, (h, c) = self.lstm(x, h)
        
        # Reshape the ouput from (samples, timesteps, output_features) to a shape appropriate for the fully connected layer
        #   (batch_size * timesteps , hidden_size)
        out = out.reshape(out.size(0) * out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h,c)

In [16]:
model = TextGenerator(len(vocab), embed_size, hidden_size, num_layers)

In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
for epoch in range(num_epochs):
    # set initial hidden and cell states
    states = (
        torch.zeros(num_layers, batch_size, hidden_size),
        torch.zeros(num_layers, batch_size, hidden_size)
    )
    
    for i in range(0, rep_tensor.size(1) - timesteps, timesteps):
        inputs = rep_tensor[:, i:i+timesteps]
        target = rep_tensor[:, (i+1):(i+1)+timesteps]
        # String: Black Horse is here
        # input : Black Horse
        # Output : lack Horse i
        outputs, _ = model(inputs, states)
        loss = loss_fn(outputs, target.reshape(-1))
        
        # Back propagation and weight update
        model.zero_grad()
        loss.backward()
        
        # perform Gradient Clipping. clip_value(float or int)
        #   Gradient Clipping will prevent the number getting to big
        #   The gradients are clipped in the range [-clip_value, clip_value].
        #   This will prevent the exploding gradients problem
        clip_grad_norm(model.parameters(), 0.5)
        optimizer.step()
        
        step = (i+1) // timesteps
        if step % 100 == 0:
            print('Epoch [{}/{}], Loss : {:.4f}'.format(epoch+1, num_epochs, loss.item()))

C:\Users\jarka\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch [1/20], Loss : 8.5755
Epoch [2/20], Loss : 6.0056
Epoch [3/20], Loss : 5.2900
Epoch [4/20], Loss : 4.8868
Epoch [5/20], Loss : 4.3771
Epoch [6/20], Loss : 3.9327
Epoch [7/20], Loss : 3.5287
Epoch [8/20], Loss : 3.0993
Epoch [9/20], Loss : 2.7470
Epoch [10/20], Loss : 2.4282
Epoch [11/20], Loss : 2.1245
Epoch [12/20], Loss : 1.8412
Epoch [13/20], Loss : 1.5435
Epoch [14/20], Loss : 1.2996
Epoch [15/20], Loss : 1.0660
Epoch [16/20], Loss : 0.8774
Epoch [17/20], Loss : 0.6061
Epoch [18/20], Loss : 0.4166
Epoch [19/20], Loss : 0.2768
Epoch [20/20], Loss : 0.1559


In [26]:
# Test the model
with torch.no_grad(): # Make sure no backpropgation
    with open('results.txt','w') as f:
        # set initial hidden and cell states
        # batch size becomes 1 because we are not training it.
        states = (
            torch.zeros(num_layers, 1, hidden_size), 
            torch.zeros(num_layers, 1, hidden_size)
        ) 
        # Select one word randomly and convert it to shape (1,1)
        inputs = torch.randint(0, len(vocab), (1,)).long().unsqueeze(1)

        for i in range(500):
            output, _ = model(inputs, states)
            print(output.shape)
            # sample a word id from the exponential of the output
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()
            print(word_id)
            # Replace the input with sampled word id for the next timestep
            inputs.fill_(word_id)

            # Write the results to the file
            word = corpus.dictionary.indexToWord[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i + 1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, 500, 'results.txt'))

torch.Size([1, 5290])
5
torch.Size([1, 5290])
9
torch.Size([1, 5290])
98
torch.Size([1, 5290])
217
torch.Size([1, 5290])
218
torch.Size([1, 5290])
13
torch.Size([1, 5290])
3
torch.Size([1, 5290])
4772
torch.Size([1, 5290])
5
torch.Size([1, 5290])
898
torch.Size([1, 5290])
781
torch.Size([1, 5290])
44
torch.Size([1, 5290])
4619
torch.Size([1, 5290])
34
torch.Size([1, 5290])
2430
torch.Size([1, 5290])
3
torch.Size([1, 5290])
3968
torch.Size([1, 5290])
129
torch.Size([1, 5290])
114
torch.Size([1, 5290])
3
torch.Size([1, 5290])
3161
torch.Size([1, 5290])
3209
torch.Size([1, 5290])
627
torch.Size([1, 5290])
645
torch.Size([1, 5290])
167
torch.Size([1, 5290])
3
torch.Size([1, 5290])
4772
torch.Size([1, 5290])
5
torch.Size([1, 5290])
5
torch.Size([1, 5290])
5
torch.Size([1, 5290])
5
torch.Size([1, 5290])
22
torch.Size([1, 5290])
28
torch.Size([1, 5290])
35
torch.Size([1, 5290])
2678
torch.Size([1, 5290])
1377
torch.Size([1, 5290])
5
torch.Size([1, 5290])
5
torch.Size([1, 5290])
1981
torch.Siz

1001
torch.Size([1, 5290])
103
torch.Size([1, 5290])
3187
torch.Size([1, 5290])
3
torch.Size([1, 5290])
5239
torch.Size([1, 5290])
3556
torch.Size([1, 5290])
5
torch.Size([1, 5290])
439
torch.Size([1, 5290])
209
torch.Size([1, 5290])
3
torch.Size([1, 5290])
915
torch.Size([1, 5290])
27
torch.Size([1, 5290])
1609
torch.Size([1, 5290])
76
torch.Size([1, 5290])
103
torch.Size([1, 5290])
3307
torch.Size([1, 5290])
5
torch.Size([1, 5290])
3679
torch.Size([1, 5290])
20
torch.Size([1, 5290])
3
torch.Size([1, 5290])
3161
torch.Size([1, 5290])
3209
torch.Size([1, 5290])
627
torch.Size([1, 5290])
645
torch.Size([1, 5290])
852
torch.Size([1, 5290])
645
torch.Size([1, 5290])
1382
torch.Size([1, 5290])
1383
torch.Size([1, 5290])
34
torch.Size([1, 5290])
87
torch.Size([1, 5290])
27
torch.Size([1, 5290])
28
torch.Size([1, 5290])
35
torch.Size([1, 5290])
2678
torch.Size([1, 5290])
334
torch.Size([1, 5290])
44
torch.Size([1, 5290])
4441
torch.Size([1, 5290])
55
torch.Size([1, 5290])
5
torch.Size([1, 52